# The non-linear Kirchhoff model

## Model and energy

## The isometry constraint

## Discretisation

## Linearisation of the nodal isometry constraint

> For this, it is important ot realize that for the empleyed finite element space $W_h$, the nodal values of the discrete deformation ($u_h(z) : z \in \mathcal{N}_h$) are mutually independent variables in the minimization problem.

## Discrete $H^2$ gradient flow

# Implementation


Besides the work already done for [the linear case](linear-kirchhoff.ipynb), we need to compute the matrix $B_{k-1}$ which enforces the nodal isometry constraint for the solution $d_t y^k$ at step $k$.

The system to solve is (Bartels, 2013):

$$\left(\begin{array}{cc}
  (1 + \alpha \tau) M^{\top} A^{(2)} M & B^{\top}_{k - 1}\\
  B_{k - 1} & I
\end{array}\right)  \left(\begin{array}{c}
  d_t Y^k\\
  \Lambda
\end{array}\right) = \left(\begin{array}{c}
  - \alpha M^{\top} A^{(2)} MY^{k - 1} + F\\
  0
\end{array}\right)$$

The local tensor matrix requires then the discrete gradient matrix $M$, the local tensor for the form $(\nabla u, \nabla v)$ in $P_2$ and the constraints matrix $B_{k-1}$. Note also that we now have vector valued functions and this will probably require tweaking some of the previous code.

## Discrete isometry constraint

\\[ \left( \begin{array}{ccccccccc}
     \quad & 2 y^{k - 1}_{i, 1} &  & \quad & 2 y^{k - 1}_{i, 1} &  & \quad & 2
     y^{k - 1}_{i, 1} & \\
     & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k -
     1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1}\\
     & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k -
     1}_{i, 1} &  & y^{k - 1}_{i, 2} & y^{k - 1}_{i, 1}\\
     &  & 2 y^{k - 1}_{i, 2} &  &  & 2 y^{k - 1}_{i, 2} &  &  & 2 y^{k -
     1}_{i, 2}
   \end{array}\right)  \left(\begin{array}{c}
     d_t y^k_i\\
     d_t y^k_{i, 1}\\
     d_t y^k_{i, 2}\\
     d_t y^k_i\\
     d_t y^k_{i, 1}\\
     d_t y^k_{i, 2}\\
     d_t y^k_i\\
     d_t y^k_{i, 1}\\
     d_t y^k_{i, 2}
   \end{array}\right) \\]

# Tests

In [ ]:
from dolfin import *
import numpy as np
np.set_printoptions(precision=4, linewidth=130, threshold=5000, suppress=True)

import nbimporter
from interpolation import interpolate

import matplotlib.pyplot as pl
%matplotlib inline

#info(parameters, True)

def bitmap(A, rtol=1e-5, atol=1e-8, figsize=(10,10), cmap='binary', **kwargs):
    """Draw the number of non zeros of A."""
    if np.sum(A.shape) > 4000:
        print("Matrix is too big (%d x %d)" % A.shape)
    pl.figure(figsize=figsize)
    bmap = 1 - np.isclose(A, 0, rtol, atol).astype(np.int)
    pl.imshow(bmap, cmap=cmap, **kwargs)
    print("%.2f%% non zeros" % (100*bmap.sum()/np.product(bmap.shape)))

In [ ]:
B0 = np.loadtxt("/home/fenics/local/src/nonlinear-kirchhoff/build/B0.txt")
A = np.loadtxt("/home/fenics/local/src/nonlinear-kirchhoff/build/A.txt")
Y0 = np.loadtxt("/home/fenics/local/src/nonlinear-kirchhoff/build/y0.txt")
Yk = np.loadtxt("/home/fenics/local/src/nonlinear-kirchhoff/build/yk.txt")
dtY = np.loadtxt("/home/fenics/local/src/nonlinear-kirchhoff/build/dtY_L.txt")
P26 = np.loadtxt("/home/fenics/local/src/nonlinear-kirchhoff/build/P26.txt")
# Cut out the bottom extra chunk in the solution
dtY = dtY[:-13]

In [ ]:
A.shape, B0.shape, Y0.shape, Yk.shape, dtY.shape, P26.shape

## Exploring the stiffness matrix

First a nice picture:

In [ ]:
bitmap(A, figsize=(5,5), interpolation='none')

In [ ]:
bitmap(B0, figsize=(10,4), interpolation='none')

The condition nunmber is **awful**, something is definitely wrong:

In [ ]:
np.linalg.cond(A)

## Visualizing the components of the solution

A quick visualization skipping Paraview...

Recall that the $W^3$ space has 27 dofs in 3 groups of 9, one per subsp
ace. Inside each group the dofs are: [evaluation, eval of dx, eval of dy] at each of the 3 vertices in turn.

In [ ]:
W = VectorFunctionSpace(UnitSquareMesh(1,1, "crossed"), "DKT", degree=3, dim=3)
y0 = Function(W)
yk = Function(W)

In [ ]:
y0.vector().set_local(Y0)
_ = yk.vector().set_local(Yk)

W.mesh().num_cells()

In [ ]:
_xx = np.array(sorted(W.mesh().coordinates()[:,0]))

pl.plot(_xx, [y0(x,0.5)[0] - x for x in _xx], label='$u_0$')
for _y in np.linspace(0,1,5):
    pl.plot(_xx, [yk(x,_y)[0] - x for x in _xx], label='$u_k(%.1f)$' % _y)
_ = pl.legend()

In [ ]:
pl.figure(figsize=(12,6))
pl.subplot(1,2,1)
plot_component(Y0, W, 1, dfdx)
pl.subplot(1,2,2)
plot_component(Y0, W, 2, dfdy)

All iterates $y^k$ fulfill the boundary condition since updates are always made with $d_tY$ with zero value and derivative at the boundary. We expect the same result as above:

In [ ]:
np.allclose(Yk[dfdx[0]], 1), np.allclose(Yk[dfdy[1]], 1)

Something is clearly off, let's plot it:

In [ ]:
from dofs import plot_dofs, plot_field_at_dofs

In [ ]:
pl.figure(figsize=(12,6))
pl.subplot(1,2,1)
plot_dofs(W, np.where(np.logical_not(np.isclose(Yk[dfdx[0]], 1)))[0])
pl.subplot(1,2,2)
plot_dofs(W, np.where(np.logical_not(np.isclose(Yk[dfdy[0]], 0)))[0])

## Inspecting the $P_2^{3 \times 2}$ tensor

Plots of the two local tensors for a 1x1 grid. What are those spurious entries on the top right corner?

In [ ]:
bitmap(P26, figsize=(5,5), cmap='Purples', interpolation='none')

```c++
  // position in array is destination row, value is source row:
  int permutations[] = {0,6,1,7,2,8,3,9,4,10,5,11};
```

In [ ]:
T = TensorFunctionSpace(UnitSquareMesh(1,1), "Lagrange", 2, shape=(3,2))
for sub in range(6):
    print(T.sub(sub).dofmap().cell_dofs(0))

dm = T.dofmap()
dm.cell_dofs(0)

This replicates the contents of the UFL file:

In [ ]:
domain = W.mesh()
T = TensorFunctionSpace(domain, "Lagrange", 2, shape=(3,2))
#W = VectorFunctionSpace(domain, "DKT", 3, dim=3)
P = VectorFunctionSpace(domain, "Lagrange", 3, dim=3)

p = TrialFunction(T)
q = TestFunction(T)
p22 = inner(nabla_grad(p), nabla_grad(q))*dx

u = TrialFunction(W)
v = TestFunction(W)
dkt = inner(u,v)*dx

f = Coefficient(W)
force = inner(f,v)*dx

# Define variational problem for projection
g = Coefficient(P)
Pg = TrialFunction(W)
w = TestFunction(W)
project_lhs = inner(w, Pg)*dx
project_rhs = inner(w, g)*dx

In [ ]:
A = assemble(p22)
A.array().shape

In [ ]:
bitmap(A.array())

### The local tensor on a 1x1 grid

Plots of the two local tensors for a 1x1 grid. What are those spurious entries on the top right corner?

In [ ]:
Alocbmap = (np.round(Aloc, 5) != 0).astype(np.int)
pl.imshow(Alocbmap)

In [ ]:
Aloc2bmap = (np.round(Aloc2, 5) != 0).astype(np.int)
pl.imshow(Aloc2bmap)